# Dependencies

In [20]:
# ! pip install --upgrade langchain-groq neo4j pip install wikipedia tiktoken yfiles_jupyter_graphs

# POC

In [1]:
# Load ENV VARS
from dotenv import find_dotenv, load_dotenv
import os
load_dotenv(find_dotenv())

GROQ_API_KEY = os.environ.get("GROQ_API_KEY")

In [3]:
from langchain_groq import ChatGroq
GROQ_MODEL_NAME = "llama-3.2-11b-text-preview"
llm = ChatGroq(groq_api_key=GROQ_API_KEY,name=GROQ_MODEL_NAME)
llm

ChatGroq(name='llama-3.2-11b-text-preview', client=<groq.resources.chat.completions.Completions object at 0x7fbce357f1f0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7fbce357fee0>, model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
from langchain_core.documents import Document

text = """
Mr. and Mrs. Dursley, of number four, Privet
Drive, were proud to say that they were perfectly
normal, thank you very much. Mr. Dursley made
drills. He was a big, beefy man with hardly any
neck, although he did have a very large moustache.
Mrs. Dursley was thin and blonde and had twice
the usual amount of neck, which came in very
useful as she spent so much of her time spying on the
neighbours. The Dursleys had a small son called
Dudley and in their
opinion there was no finer boy anywhere.
Mrs Dursley had a sister called Lily Potter. She
and her husband James Potter had a son called
Harry Potter. They lived far from the Dursleys and
did not speak to them much. They did not get
along.
One day, a man appeared outside of
the Dursleys house. He was tall, thin, and very
old, judging by the silver of his hair and beard,
which were both long enough to tuck into his belt.
He was wearing long robes,
a purple cloak that swept the ground, and highheeled, buckled boots. His blue eyes were light,
bright, and sparkling behind half-moon spectacles
and his nose was very long and crooked, as though
it had been broken at least twice. This man's name
was Albus Dumbledore. 
"""

documents = [Document(page_content=text)]
print(documents)

[Document(metadata={}, page_content="\nMr. and Mrs. Dursley, of number four, Privet\nDrive, were proud to say that they were perfectly\nnormal, thank you very much. Mr. Dursley made\ndrills. He was a big, beefy man with hardly any\nneck, although he did have a very large moustache.\nMrs. Dursley was thin and blonde and had twice\nthe usual amount of neck, which came in very\nuseful as she spent so much of her time spying on the\nneighbours. The Dursleys had a small son called\nDudley and in their\nopinion there was no finer boy anywhere.\nMrs Dursley had a sister called Lily Potter. She\nand her husband James Potter had a son called\nHarry Potter. They lived far from the Dursleys and\ndid not speak to them much. They did not get\nalong.\nOne day, a man appeared outside of\nthe Dursleys house. He was tall, thin, and very\nold, judging by the silver of his hair and beard,\nwhich were both long enough to tuck into his belt.\nHe was wearing long robes,\na purple cloak that swept the ground

In [11]:
from langchain_experimental.graph_transformers import LLMGraphTransformer

llm_transformer = LLMGraphTransformer(llm=llm)

graph_documents = llm_transformer.convert_to_graph_documents(documents=documents)

In [18]:
graph_documents[0]

GraphDocument(nodes=[], relationships=[], source=Document(metadata={}, page_content="\nMr. and Mrs. Dursley, of number four, Privet\nDrive, were proud to say that they were perfectly\nnormal, thank you very much. Mr. Dursley made\ndrills. He was a big, beefy man with hardly any\nneck, although he did have a very large moustache.\nMrs. Dursley was thin and blonde and had twice\nthe usual amount of neck, which came in very\nuseful as she spent so much of her time spying on the\nneighbours. The Dursleys had a small son called\nDudley and in their\nopinion there was no finer boy anywhere.\nMrs Dursley had a sister called Lily Potter. She\nand her husband James Potter had a son called\nHarry Potter. They lived far from the Dursleys and\ndid not speak to them much. They did not get\nalong.\nOne day, a man appeared outside of\nthe Dursleys house. He was tall, thin, and very\nold, judging by the silver of his hair and beard,\nwhich were both long enough to tuck into his belt.\nHe was wearing l

# Graph RAG (using neo4j)

In [ ]:
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Tuple, List, Optional
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
import os
from langchain_community.graphs import Neo4jGraph
from langchain.document_loaders import WikipediaLoader
from langchain.text_splitter import TokenTextSplitter
from langchain_groq import ChatGroq
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector

from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_core.runnables import ConfigurableField, RunnableParallel, RunnablePassthrough
